In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from visualisation.utils import *
from visualisation.sobol import *
from visualisation.plotsobol_cprice import *
from visualisation.emissionpathways import *

In [2]:
from carbontaxdamages.defaultparams import Params
from carbontaxdamages.run import full_run_structured, export_output
from carbontaxdamages.economics import *

In [3]:
dataset = 'Default'  # One of ['Default', 'No inertia', 'Drupp']

## Import data

In [4]:
param_columns = ['SSP', 'damage', 'TCRE', 'cost_level', 'r']
r_values = {'0.1%': 0.001, '1.5%': 0.015, '3%': 0.03}
default_r = '1.5%'
default_elasmu = 'all'

# By default, make sure PRTP has uniform distribution
if 'r' in cost_and_TCRE_probs:
    del(cost_and_TCRE_probs['r'])

if dataset == 'Default':
    outputfolder = 'default'
    df, df_indexed = parse_input('experiment_allcba.csv', withInertia=True)
    df_CE, df_CE_indexed = parse_input('experiment_allcarbonbudget.csv', withInertia=True)
    
if dataset == 'No inertia':
    outputfolder = 'no_inertia'
    df, df_indexed = parse_input('experiment_allcba.csv', withInertia=False)
    df_CE, df_CE_indexed = parse_input('experiment_allcarbonbudget.csv', withInertia=False)
    
if dataset == 'Drupp':
    outputfolder = 'drupp'
    custom_map_names = dict(map_names)
    df, df_indexed = parse_input('experiment_allcbadrupp.csv', allElasmu=True, withInertia=False)
    df_CE, df_CE_indexed = parse_input('experiment_alldruppcarbonbudget.csv', allElasmu=True, withInertia=False)
    
    r_values = {'0.0_0.5': 0.0, '0.0_1.5': 0.0, '0.02_2.5': 0.02}
    default_r = '0.0_1.5'
    for _df in [df, df_CE]:
        _df['r'] = pd.Categorical(_df['r'].astype(str)+'_'+_df['elasmu'], list(r_values.keys()))
        _df['name'] = _df['name']+'_'+_df['elasmu']
        
    # Update the probabilities:
    cost_and_TCRE_probs['r'] = {
        'values': list(r_values.keys()),
        'p': cost_and_TCRE_probs['cost_level']['p']
    } 


## Individual factors

In [20]:
lowdamage = 'No damages'

fig1 = plot_individual(df_CE, 'cost_level', letter='a', legend_title='Cost level', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu)
fig1.show()

fig2 = plot_individual(df_CE, 'SSP', ['SSP1', 'SSP3', 'SSP5'], 3, letter='b', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu)
fig2.show()

fig3 = plot_individual(df_CE, 'TCRE', 'all', 6, letter='c', legend_title='TCRE', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu)
fig3.show()

fig4 = plot_individual(df_CE, param_columns[-1], 'all', 11, letter='d', legend_title='PRTP', lowdamage=lowdamage, elasmu=default_elasmu)
fig4.show()

In [9]:
fig1.write_image(f'../Paper/img/{outputfolder}/CE_individual_paths_a.png', scale=2)
fig2.write_image(f'../Paper/img/{outputfolder}/CE_individual_paths_b.png', scale=2)
fig3.write_image(f'../Paper/img/{outputfolder}/CE_individual_paths_c.png', scale=2)
fig4.write_image(f'../Paper/img/{outputfolder}/CE_individual_paths_d.png', scale=2)

### Min emission level

In [25]:
fig1.update_traces(showlegend=False, line_color='rgba(0,0,0,0.15)')#.data

In [30]:
df_CE_minEmissions, df_CE_minEmissions_indexed = parse_input(
    'experiment_extra.csv', allElasmu=True, withInertia=True
)
lowdamage='No damages'

fig1_minEmissions = fig1.update_traces(showlegend=False, line_color='rgba(0,0,0,0.1)')
for trace in plot_individual(df_CE_minEmissions, 'cost_level', letter='a', legend_title='Cost level', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu).data:
    fig1_minEmissions.add_trace(trace)
fig1_minEmissions.show()

fig2_minEmissions = fig2.update_traces(showlegend=False, line_color='rgba(0,0,0,0.1)')
for trace in plot_individual(df_CE_minEmissions, 'SSP', ['SSP1', 'SSP3', 'SSP5'], 3, letter='b', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu).data:
    fig2_minEmissions.add_trace(trace)
fig2_minEmissions.show()

fig3_minEmissions = fig3.update_traces(showlegend=False, line_color='rgba(0,0,0,0.1)')
for trace in plot_individual(df_CE_minEmissions, 'TCRE', 'all', 6, letter='c', legend_title='TCRE', lowdamage=lowdamage, r=default_r, elasmu=default_elasmu).data:
    fig3_minEmissions.add_trace(trace)
fig3_minEmissions.show()

fig4_minEmissions = fig4.update_traces(showlegend=False, line_color='rgba(0,0,0,0.1)')
for trace in plot_individual(df_CE_minEmissions, param_columns[-1], 'all', 11, letter='d', legend_title='PRTP', lowdamage=lowdamage, elasmu=default_elasmu).data:
    fig4_minEmissions.add_trace(trace)
fig4_minEmissions.show()

fig1_minEmissions.write_image(f'../Paper/img/{outputfolder}/minEmissions_CE_individual_paths_a.png', scale=2)
fig2_minEmissions.write_image(f'../Paper/img/{outputfolder}/minEmissions_CE_individual_paths_b.png', scale=2)
fig3_minEmissions.write_image(f'../Paper/img/{outputfolder}/minEmissions_CE_individual_paths_c.png', scale=2)
fig4_minEmissions.write_image(f'../Paper/img/{outputfolder}/minEmissions_CE_individual_paths_d.png', scale=2)

### Increased initial carbon price compared to no-damages

In [15]:
year = 2025

try:
    _tmp_df = df_CE_indexed
    values = [
        [
            _tmp_df.loc[('SSP2', 'No damages', '0.62', '5th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', '5th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'DICE', '0.62', '5th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', '5th perc.', '1.5%', '2.0', '1.001', year), 'p']
        ],
        [
            _tmp_df.loc[('SSP2', 'No damages', '0.62', 'Median', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', 'Median', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'DICE', '0.62', 'Median', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', 'Median', '1.5%', '2.0', '1.001', year), 'p']
        ],
        [
            _tmp_df.loc[('SSP2', 'No damages', '0.62', '95th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', '95th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'DICE', '0.62', '95th perc.', '1.5%', '2.0', '1.001', year), 'p'],
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', '95th perc.', '1.5%', '2.0', '1.001', year), 'p']
        ]
    ]
    print("                         [costs: 5th, 50th, 95th percentiles]\n")
    print("HowardTotal vs nodamage:", [v[1]/v[0] for v in values])
    print("DICE vs nodamage:", [v[2]/v[0] for v in values])
    print("Burke (LR) vs nodamage:", [v[3]/v[0] for v in values])
    
except KeyError as e:
    print("Key does not exist:", e)

                         [costs: 5th, 50th, 95th percentiles]

HowardTotal vs nodamage: [2.8191461473025825, 1.7595811975026976, 1.319228349530206]
DICE vs nodamage: [1.4635523501068186, 1.1724837667344852, 1.0794810220533124]
Burke (LR) vs nodamage: [3.1049252128754348, 2.7114132193248377, 1.9032880224402038]


And for the discount rate:

In [13]:

try:
    values = [
        [
            _tmp_df.loc[('SSP2', 'No damages', '0.62', 'Median', '0.1%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'No damages', '0.62', 'Median', '1.5%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'No damages', '0.62', 'Median', '3%', '2.0', '1.45', year), 'p']
        ],
        [
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', 'Median', '0.1%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', 'Median', '1.5%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'Howard Total', '0.62', 'Median', '3%', '2.0', '1.45', year), 'p']
        ],
        [
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', 'Median', '0.1%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', 'Median', '1.5%', '2.0', '1.45', year), 'p'],
            _tmp_df.loc[('SSP2', 'Burke (LR)', '0.62', 'Median', '3%', '2.0', '1.45', year), 'p']
        ]
    ]
    print("0.1% vs 1.5%:", [v[0]/v[1] for v in values])
    print("1.5% vs 3%:", [v[1]/v[2] for v in values])
    print("0.1% vs 3%:", [v[0]/v[2] for v in values])
    
except KeyError as e:
    print("Key does not exist:", e)

Key does not exist: ('SSP2', 'No damages', '0.62', 'Median', '0.1%', '2.0', '1.45', 2025)


## Sobol analysis on 2030/2050/2070 price

In [32]:
years = np.arange(2030, 2101, 5)

split_param = 'cost_level'
subplot_titles = (
    'All mitigation cost levels', 
    'Mitig. cost level: 5th perc.', 'Mitig. cost level: median', 'Mitig. cost level: 95th perc.'
)

In [6]:
for relative in [True, False]:
    sobol_CE_per_year = calculate_sobol_cprices(df_CE, split_param, relative, param_columns, years, cost_and_TCRE_probs, num=10**6)
    
    for use_sqrt in [True, False]:
        if relative and use_sqrt:
            continue
            
        fig_sobol_cprice_CE = plotsobol_cprice(sobol_CE_per_year, use_sqrt, subplot_titles, relative, param_columns, years)
        
        if relative:
            suffix = '_relative'
        else:
            suffix = '_absolute_sqrt' if use_sqrt else '_absolute_variance'

        fig_sobol_cprice_CE.write_image(f'../Paper/img/{outputfolder}/CE_partial_uncertainty{suffix}.png', scale=2)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [01:32<00:00,  6.19s/it]


In [33]:
sobol_CE_per_year_nodmg = calculate_sobol_cprices(
    df_CE, split_param, False, param_columns, years, 
    {'cost_level': cost_and_TCRE_probs['cost_level']}, num=10**6, valid_dmg_fcts=['No damages']
)
fig_sobol_cprice_CE_nodmg = plotsobol_cprice(
    sobol_CE_per_year_nodmg, False, subplot_titles, False, param_columns, years
)
fig_sobol_cprice_CE_nodmg.write_image(f'../Paper/img/{outputfolder}/CE_partial_uncertainty_absolute_variance_nodmg.png', scale=2)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [01:23<00:00,  5.53s/it]


## Mitigation costs vs damage costs

In [15]:
damage_functions_2 = {
    'No damages': damages['nodamage'],
    'DICE': damages['damageDICE'],
    'Burke (LR)': damages['damageBurkeWithLag']['SSP2'],
    'Howard Total': damages['damageHowardTotal']
}

In [16]:
T0 = 1.0 + 4 * 0.00062 * 38.8

def mitigation_vs_damages(
    SSP='SSP2', 
    r=default_r, 
    cost_level='Median', 
    TCRE='0.62', 
    damage='Howard Total', 
    elasmu=default_elasmu,
    max_y=0.149, title=''
):
    
    selection1 = df_CE[
        (df_CE['SSP'] == SSP) & (df_CE['r'] == r) & (df_CE['cost_level'] == cost_level)
        & ((df_CE['elasmu'] == elasmu) | (elasmu == 'all')) & (df_CE['beta'] == '2.0')
        & (df_CE['TCRE'] == TCRE) & (df_CE['damage'] == damage)
    ]

    selection1b = df_CE[
        (df_CE['SSP'] == SSP) & (df_CE['r'] == r) & (df_CE['cost_level'] == cost_level)
        & ((df_CE['elasmu'] == elasmu) | (elasmu == 'all')) & (df_CE['beta'] == '2.0')
        & (df_CE['TCRE'] == TCRE) & (df_CE['damage'] == 'No damages')
    ]

    selection2 = df[
        (df['SSP'] == SSP) & (df['r'] == r) & (df['cost_level'] == cost_level)
        & ((df['elasmu'] == elasmu) | (elasmu == 'all')) & (df['beta'] == '2.0')
        & (df['TCRE'] == TCRE) & (df['damage'] == damage)
    ]

    t_values_years = np.linspace(2020, 2150, 27)
    dt = t_values_years[1] - t_values_years[0]
    temp_values = np.cumsum(baseline_emissions(t_values_years, SSP) * dt) * float(TCRE) / 1000 + T0
    
    T_2100_baseline = np.interp(2100, t_values_years, temp_values)
    T_2100_with = selection1[selection1['year'] == 2100]['temp'].values[0]
    T_2100_without = selection2[selection2['year'] == 2100]['temp'].values[0]
    
    fig = make_subplots(1, 3, subplot_titles=(
        'Baseline<br>(T<sub>2100</sub>: {:.1f}°C)'.format(T_2100_baseline),
        'With target<br>(T<sub>2100</sub>: {:.1f}°C)'.format(T_2100_with),
        'Without target<br>(T<sub>2100</sub>: {:.1f}°C)'.format(T_2100_without)
    ), shared_yaxes=True)
    

    fig.append_trace(go.Scatter(x=t_values_years, y=[damage_functions_2[damage](T) for T in temp_values], stackgroup='baseline', fillcolor=colors_PBL[0], line_width=0, showlegend=False, legendgroup='damages'), 1, 1)

    fig.append_trace(go.Scatter(x=selection1['year'], y=selection1['abatementFraction'], stackgroup='withtarget', fillcolor=colors_PBL[2], line_width=0, name='Mitigation costs', legendgroup='mitigation'), 1, 2)
    fig.append_trace(go.Scatter(x=selection1['year'], y=selection1['damageFraction'], stackgroup='withtarget', fillcolor=colors_PBL[0], line_width=0, name='Damages', legendgroup='damages'), 1, 2)

    fig.append_trace(go.Scatter(x=selection1b['year'], y=selection1b['abatementFraction'], line_color='#000', line_dash='dot', name='Mitigation costs<br>(cost effective)', legendgroup='costeffective'), 1, 2)

    fig.append_trace(go.Scatter(x=selection2['year'], y=selection2['abatementFraction'], stackgroup='withouttarget', fillcolor=colors_PBL[2], line_width=0, showlegend=False, legendgroup='mitigation'), 1, 3)
    fig.append_trace(go.Scatter(x=selection2['year'], y=selection2['damageFraction'], stackgroup='withouttarget', fillcolor=colors_PBL[0], line_width=0, showlegend=False, legendgroup='damages'), 1, 3)

    fig.update_xaxes(range=[2021,2100])
    fig.update_xaxes(range=[2021,2100], col=3)
    fig.update_yaxes(range=[0, max_y], tickformat=',.0%')
    fig.update_layout(
        legend_y=0.5, margin={'t': 110, 'b': 0, 'l': 30},
        width=950, height=320, yaxis1={'title': 'Share of world GDP', 'title_standoff': 10},
        title={'text':title, 'x': 0}
    )
    
    return fig

fig1 = mitigation_vs_damages(cost_level='5th perc.', damage='Burke (LR)', title='<b>a. Low costs, damage: Burke (LR)</b>')
fig1.show()
fig2 = mitigation_vs_damages(cost_level='Median', damage='Howard Total', title='<b>b. Medium costs, damage: Howard Total</b>')
fig2.show()
fig3 = mitigation_vs_damages(cost_level='95th perc.', damage='DICE', title='<b>c. High costs, damage: DICE</b>')
fig3.show()

In [17]:
fig1.write_image(f'../Paper/img/{outputfolder}/mitigation_vs_damages_a.png', scale=2)
fig2.write_image(f'../Paper/img/{outputfolder}/mitigation_vs_damages_b.png', scale=2)
fig3.write_image(f'../Paper/img/{outputfolder}/mitigation_vs_damages_c.png', scale=2)

## Mitigation costs vs avoided damages

In [17]:
damage_functions = {
    'No damages': damages['nodamage'],
    'DICE': damages['damageDICE'],
    'Burke (LR)': damages['damageBurkeWithLag'],
    'Howard Total': damages['damageHowardTotal']
}

In [18]:
def baseline_damages(SSP, damage, TCRE, t_values_years):
    T0 = Params().default_params['T0']
    dt = t_values_years[1] - t_values_years[0]
    temp_values = np.cumsum(baseline_emissions(t_values_years, SSP) * dt) * float(TCRE) / 1000 + T0
    dmg_fct = damage_functions[damage][SSP] if damage == 'Burke (LR)' else damage_functions[damage]
    return [dmg_fct(T) for T in temp_values]

def calcBaselineDamages(sub_df):
    SSP,damage,TCRE = sub_df.iloc[0][['SSP','damage','TCRE']]
    sub_df['baselineDamages'] = baseline_damages(SSP, damage, TCRE, sub_df['year'].values)
    return sub_df

def npv_series(selection, name):
    selection['{} NPV'.format(name)] = selection[name] * np.exp(-selection['r_num'] * (selection['year'] - 2020))
    #return discounted_values.groupby(param_columns).sum()
    
def calc_costs_vs_benefits(selection):
    selection['damage'] = pd.Categorical(selection['damage'], selection['damage'].unique())
    selection['baselineDamages'] = np.nan
    
    selection = selection.groupby(param_columns).apply(calcBaselineDamages)
    
    selection['abatement_costs'] = selection['abatementFraction'] * selection['Ygross']
    selection['remaining_damage_costs'] = selection['damageFraction'] * selection['Ygross']
    selection['avoided_damages'] = selection['baselineDamages'] * selection['Ygross'] - selection['remaining_damage_costs']
    
    selection['r_num'] = selection['r'].replace(r_values)
    
    npv_series(selection, 'abatement_costs')
    npv_series(selection, 'avoided_damages')
    npv_series(selection, 'Y')
    
    NPV_values = selection.groupby(param_columns)[['abatement_costs NPV', 'avoided_damages NPV', 'Y NPV']].sum().reset_index()
    NPV_values['abatement_costs NPV relative'] = NPV_values['abatement_costs NPV'] / NPV_values['Y NPV']
    NPV_values['avoided_damages NPV relative'] = NPV_values['avoided_damages NPV'] / NPV_values['Y NPV']
    
    NPV_values['damage'] = pd.Categorical(NPV_values['damage'], ['DICE', 'Howard Total', 'Burke (LR)'])
    NPV_values = NPV_values.sort_values(param_columns)
    
    return NPV_values

In [19]:
selection_CBA = df.loc[
    (df['damage'].isin(['DICE', 'Burke (LR)', 'Howard Total']))
    & (df['year'] <= 2100)
    & (df['TCRE'].isin(['0.42','0.62','0.82'])), :
].copy()

In [20]:
selection_CE = df_CE.loc[(df_CE['damage'].isin(['DICE', 'Burke (LR)', 'Howard Total'])) & (df_CE['year'] <= 2100),:].copy()

In [21]:
NPV_values_CBA = calc_costs_vs_benefits(selection_CBA)
NPV_values_CBA['type'] = 'CBA'
NPV_values_CE = calc_costs_vs_benefits(selection_CE)
NPV_values_CE['type'] = 'CE'

In [23]:
selection1 = NPV_values_CE.loc[NPV_values_CE['damage'].isin(['Howard Total', 'Burke (LR)'])]
print((
    selection1['avoided_damages NPV relative'] > 0.99 * selection1['abatement_costs NPV relative']
).mean())

selection2 = NPV_values_CE.loc[NPV_values_CE['damage'].isin(['DICE'])]
print((
    selection2['avoided_damages NPV relative'] > 0.99 * selection2['abatement_costs NPV relative']
).mean())

0.937037037037037
0.4074074074074074


In [22]:
selection1 = NPV_values_CE.loc[NPV_values_CE['damage'].isin(['Howard Total', 'Burke (LR)'])]
print((
    selection1['avoided_damages NPV relative'] > 0.99 * selection1['abatement_costs NPV relative']
).mean())

selection2 = NPV_values_CE.loc[NPV_values_CE['damage'].isin(['DICE'])]
print((
    selection2['avoided_damages NPV relative'] > 0.99 * selection2['abatement_costs NPV relative']
).mean())

0.9481481481481482
0.4


In [24]:
NPV_values = NPV_values_CE

symbols = ['circle-open', 'square', 'star']

fig = px.scatter(
    NPV_values, x='abatement_costs NPV relative', y='avoided_damages NPV relative',
    color='damage', range_x=[0,0.17], hover_data=['SSP'], symbol='cost_level', symbol_sequence=symbols, hover_name='TCRE'
)
fig.update_traces(showlegend=False)

for color, damage in zip(px.colors.qualitative.Plotly[:3][::-1], NPV_values['damage'].unique()[::-1]):
    fig.add_trace(go.Scatter(
        x=[None],y=[None],mode='markers',marker_color=color, name='Damage: {}'.format(damage), legendgroup='damage'
    ))

for symbol, cost_level in zip(symbols, NPV_values['cost_level'].unique()):
    fig.add_trace(go.Scatter(
        x=[None],y=[None],mode='markers',marker_color='#000', marker_symbol=symbol,
        name='Cost level: {}'.format(cost_level), legendgroup='cost_level'
    ))

x_values = np.linspace(0,0.3,100)
fig.add_trace(go.Scatter(x=x_values, y=x_values, name='x=y line', line_color='#BBB'))

fig.update_layout(
    legend={'tracegroupgap': 15, 'y': 0.5, 'font_size': 13, 'title': None},
    width=850, height=500, margin={'t': 20},
    xaxis={'title': '<b>Costs</b><br>(NPV Abatement costs / GDP)', 'tickformat': ',.0%', 'range': [-0.0025, 0.1092]},
    yaxis={'title': '<b>Benefits</b><br>(NPV Avoided damages / GDP)', 'tickformat': ',.0%', 'range': [-0.0212, 0.52]},
)

fig.show()

In [25]:
fig.write_image(f'../Paper/img/{outputfolder}/costs_vs_benefits_CE.png', scale=2)

In [26]:
NPV_values_combined = pd.concat((NPV_values_CE, NPV_values_CBA))
NPV_values_combined['name'] = NPV_values_combined[param_columns].agg('_'.join, axis=1)
NPV_values_combined['damage'] = pd.Categorical(NPV_values_combined['damage'], ['DICE', 'Howard Total', 'Burke (LR)'])

In [27]:
fig = px.line(
    NPV_values_combined, x='abatement_costs NPV relative', y='avoided_damages NPV relative',
    line_group='name', color='damage', facet_col='cost_level', facet_col_wrap=1
)

fig.update_traces(opacity=0.8, line_width=1)
fig.update_annotations(None,font_size=14)
fig.for_each_annotation(lambda a: a.update(text="Cost level: " + a.text.split("=")[-1]))

_fig2 = px.scatter(
    NPV_values_combined[NPV_values_combined['type'] == 'CBA'].sort_values(param_columns),
    x='abatement_costs NPV relative', y='avoided_damages NPV relative',
    color='damage', facet_col='cost_level', facet_col_wrap=1, symbol='cost_level', symbol_sequence=symbols
)
for trace in _fig2.data:
    fig.add_trace(trace)

fig.update_traces(showlegend=False)

for color, damage in zip(px.colors.qualitative.Plotly[:3][::-1], NPV_values_combined['damage'].unique()[::-1]):
    fig.add_trace(go.Scatter(
        x=[None],y=[None],mode='markers+lines',marker_color=color, name='Damage: {}'.format(damage), legendgroup='damage'
    ))

for symbol, cost_level in zip(symbols, NPV_values['cost_level'].unique()):
    fig.add_trace(go.Scatter(
        x=[None],y=[None],mode='markers',marker_color='#000', marker_symbol=symbol,
        name='Cost level: {}'.format(cost_level), legendgroup='cost_level'
    ))

for i in range(3):
    x_values = np.linspace(0,0.3,100)
    fig.add_trace(go.Scatter(x=x_values, y=x_values, line_color='#333', line_width=1, name='x=y line', showlegend=i==0), i+1, 1)
    
fig.update_layout(
    xaxis1={
        'range': [-0.01, 0.1627], 'title': '<b>Costs</b><br>(NPV Abatement costs / GDP)', 'tickformat': ',.0%'
    }, yaxis_range=[-0.02,0.52],
    height=1000, margin={'t': 20}, width=900,
    legend_tracegroupgap=15, legend_y=0.5, legend_font_size=13, legend_title=None
)
fig.update_yaxes(title='<b>Benefits</b><br>(NPV Avoided damages / GDP)', tickformat=',.0%')
fig.show()

In [28]:
fig.write_image(f'../Paper/img/{outputfolder}/costs_vs_benefits_CBA.png', scale=2)

## All carbon price paths

In [34]:
def make_price_and_emission_paths(selection, SSPs=['SSP1', 'SSP3', 'SSP5']):

    fig = make_subplots(
        2,3,
        subplot_titles=('Cost level: 5th perc.<br><br>Price', 'Cost level: median<br><br>  ', 'Cost level: 95th perc.<br><br>  ', 'Emissions'),
        vertical_spacing=0.15
    )

    for j, var in enumerate(['p', 'E']):
        for i, cost_level in enumerate(['5th perc.', 'Median', '95th perc.']):
            for SSP, color in zip(SSPs, colors_PBL):

                # For legend
                if i == 0 and j == 0:
                    fig.append_trace(go.Scatter(
                        x=[None], y=[None], mode='lines', line={'color': color},
                        name=SSP, showlegend=True, legendgroup='SSP'
                    ), 1, 1)

                alpha_var = 'r'
                for option, alpha in zip(r_values.keys(), [0.33, 0.66, 1]):

                    # For legend
                    if SSP == SSPs[0] and i == 0 and j == 0:
                        fig.append_trace(go.Scatter(
                            x=[None], y=[None], mode='lines', line={'color': blend('#000000', alpha)},
                            name='{}={}'.format(alpha_var, option), showlegend=True, legendgroup='r'
                        ), 1, 1)

                    selection_new = selection.loc[
                        (selection['cost_level'] == cost_level) &
                        (selection['SSP'] == SSP) &
                        (selection[alpha_var] == option) &
                        ((selection['elasmu'] == default_elasmu) | (default_elasmu == 'all')) &
                        (selection['beta'] == '2.0') &
                        #(df_CE['TCRE'].isin(['0.00038', '0.00062'])) &
                        ~((selection['damage'] == 'No damages') & selection['TCRE'].isin(['0.86', '0.38']))
                    ]
                    figtmp = px.line(selection_new, x='year', y=var, line_group='name')
                    figtmp.update_traces(line_color=blend(color, alpha))
                    for trace in figtmp.data:
                        fig.append_trace(trace, j+1, i+1)

    fig.update_layout(
        xaxis1={'range': [2020,2100]}, xaxis2={'range': [2020,2100]}, xaxis3={'range': [2020,2100]},
        xaxis4={'range': [2020,2100]}, xaxis5={'range': [2020,2100]}, xaxis6={'range': [2020,2100]},
        yaxis1={'range': [0, 598.3], 'title': 'Carbon price (US$)'},
        yaxis2={'range': [0, 1495]},
        yaxis3={'range': [0, 3843]},
        yaxis4={'range': [-27.01, 58.63], 'title': 'Emissions (GtCO2/yr)'},
        yaxis5={'range': [-27.01, 58.63]},
        yaxis6={'range': [-27.01, 58.63]},
        width=950,
        margin={'l': 0, 't': 75, 'b': 30},
        height=700,
        legend={'y': 0.5, 'font_size': 14}
    )
    
    return fig
    
fig_CE = make_price_and_emission_paths(df_CE[df_CE['damage'].isin(['No damages', 'DICE', 'Howard Total', 'Burke (LR)'])])
fig_CBA = make_price_and_emission_paths(df[df['damage'].isin(['DICE', 'Howard Total', 'Burke (LR)'])])
# fig_CE.show()
fig_CE.write_image(f'../Paper/img/{outputfolder}/CE_price_and_emission_paths.png', scale=2)
fig_CBA.write_image(f'../Paper/img/{outputfolder}/CBA_price_and_emission_paths.png', scale=2)